In [436]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import plotly.express as ex
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit
import math
import numpy as np

# Importing CSV Dataset

In [ ]:
#MONTHLY SUMS OF SOLAR RADIATION AND METEOROLOGICAL PARAMETERS
#
#Customer name: Solargis s.r.o.
#Issued: 2020-09-08 14:47
#
#Site name: Plataforma Solar de Almeria, Spain (ES)
#Latitude: 37.094416
#Longitude: -2.359850
#Elevation: 497.0 m a.s.l.
#http://solargis.info/imaps/#tl=Google:satellite&loc=37.094416,-2.359850&z=14 
#
#
#Output from the climate database Solargis v2.2.10
#
#Solar radiation data
#Description: data calculated from Meteosat MSG and MFG satellite data ((c) 2020 EUMETSAT) and from atmospheric data ((c) 2020 ECMWF, NOAA and NASA) by Solargis method 
#Summarization type: monthly
#Summarization period: 01/1994 - 08/2020
#Spatial resolution: 250 m
#
#Meteorological data
#Description: spatially disaggregated from CFSv2 and GFS ((c) 2020 NOAA) and ERA5 ((c) 2020 ECMWF) by Solargis method 
#Summarization type: monthly
#Summarization period: 01/1994 - 08/2020
#Spatial resolution: temperature 1 km, other meteorological parameters 33 km to 55 km
#
#Service provider: Solargis s.r.o., M. Marecka 3, Bratislava, Slovakia
#Company ID: 45 354 766, VAT Number: SK2022962766
#Registration: Business register, District Court Bratislava I, Section Sro, File 62765/B
#http://solargis.com, contact@solargis.com
#
#Disclaimer:
#Considering the nature of climate fluctuations, interannual and long-term changes, as well as the uncertainty of measurements and calculations, Solargis s.r.o. cannot take full guarantee of the accuracy of estimates. The maximum possible has been done for the assessment of climate conditions based on the best available data, software and knowledge. Solargis s.r.o. shall not be liable for any direct, incidental, consequential, indirect or punitive damages arising or alleged to have arisen out of use of the provided data. Solargis is a trade mark of Solargis s.r.o.
#
#Copyright (c) 2020 Solargis s.r.o.
#
#
#Columns:
#Year 
#Month 
#GHId - Average daily sum of global horizontal irradiation [kWh/m2]
##GHIm - Monthly sum of global horizontal irradiation [kWh/m2]
#DNId - Average daily sum of direct normal irradiation [kWh/m2]
##DNIm - Monthly sum of direct normal irradiation [kWh/m2]
#DIFd - Average daily sum of diffuse horizontal irradiation [Wh/m2]
##DIFm - Monthly sum of diffuse horizontal irradiation [Wh/m2]
#GTId - Average daily sum of global tilted irradiation [kWh/m2] (fixed inclination: 33 deg. azimuth: 180 deg., rel. row spacing: 2.6), no data value -9
##GTIm - Monthly sum of global tilted irradiation [kWh/m2] (fixed inclination: 33 deg. azimuth: 180 deg., rel. row spacing: 2.6), no data value -9
#TEMP - Average diurnal (24 hour) air temperature at 2 m [deg. C]
#WS - Average wind speed at 10 m [m/s]
#RH - Relative humidity [%]
#AP - Atmospheric pressure [hPa]
#PWAT - Precipitable water [kg/m2], no data value -9
#
#Data:

In [437]:
df = pd.read_csv('solar_radiation.csv',sep =';')
df.insert(2,'Day',1)
df.head()

,Year,Month,Day,GHId,GHIm,DNId,DNIm,DIFd,DIFm,GTId,GTIm,TEMP,WS,RH,AP,PWAT
0,1994,1,1,2.969,92.0,5.106,158.3,0.932,28.9,4.988,154.6,9.5,2.2,62.4,949.1,8.4
1,1994,2,1,3.908,109.4,5.471,153.2,1.233,34.5,5.700,159.6,10.9,2.2,57.9,944.8,8.2
2,1994,3,1,5.119,158.7,5.739,177.9,1.791,55.5,6.329,196.2,13.7,2.0,62.2,949.6,11.5
3,1994,4,1,6.552,196.6,7.521,225.6,1.698,50.9,7.075,212.2,14.8,2.2,49.3,944.4,10.0
4,1994,5,1,7.036,218.1,6.917,214.4,2.208,68.4,6.834,211.9,20.6,2.0,46.3,942.7,14.5


In [438]:
df['Date'] = pd.to_datetime(df[['Year','Month','Day']])
df.index = df['Date'].values
df.head()

,Year,Month,Day,GHId,GHIm,DNId,DNIm,DIFd,DIFm,GTId,GTIm,TEMP,WS,RH,AP,PWAT,Date
1994-01-01,1994,1,1,2.969,92.0,5.106,158.3,0.932,28.9,4.988,154.6,9.5,2.2,62.4,949.1,8.4,1994-01-01
1994-02-01,1994,2,1,3.908,109.4,5.471,153.2,1.233,34.5,5.700,159.6,10.9,2.2,57.9,944.8,8.2,1994-02-01
1994-03-01,1994,3,1,5.119,158.7,5.739,177.9,1.791,55.5,6.329,196.2,13.7,2.0,62.2,949.6,11.5,1994-03-01
1994-04-01,1994,4,1,6.552,196.6,7.521,225.6,1.698,50.9,7.075,212.2,14.8,2.2,49.3,944.4,10.0,1994-04-01
1994-05-01,1994,5,1,7.036,218.1,6.917,214.4,2.208,68.4,6.834,211.9,20.6,2.0,46.3,942.7,14.5,1994-05-01


In [439]:
solaryear = df.groupby(['Year'])['GHIm'].mean()

fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Monthly sum of solar radiation (unit : kWh/m^2)",color = 'black')
    ))

fig.add_trace(go.Scatter(
                x=df.index,
                y=df['GHIm'],
                name="Global horizontal irradiation",
                line_color='red',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=[i for i in range(1994,2020)],
                y=solaryear,
                name="Yearly average",
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))

fig.update_layout(title_text = "Evolution of the global horizontal irradiance between 1994 and 2020", title_x=0.5, \
                  title_font_size = 22)
fig.show()

# Data approximation

In [440]:
y=df['GHIm'].values
X = np.arange(0, len(y))

In [441]:
def sinusoid(x,A,offset,omega,phase):
    R = []
    for i in x:
        R.append(A*np.sin(omega*i+phase) + offset)
    return R

T = 12 #number of month per year

def get_p0(x, y):
    A0 = (max(y[0:T]) - min(y[0:T]))/2
    offset0 = y[0]
    phase0 = 0
    omega0 = 2.*np.pi/T
    return [A0, offset0,omega0, phase0]

param, covariance = curve_fit(sinusoid, X, y, p0=get_p0(X,y))
A,offset,omega,phase = param
approx = sinusoid(X, A,offset,omega,phase)

fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Monthly sum of solar radiation (unit : kWh/m^2)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=df.index,
                y=df['GHIm'],
                name="Global horizontal irradiation",
                line={'dash': 'dash', 'color': 'red'},
                opacity=0.8))


fig.add_trace(go.Scatter(
                x=df.index,
                y=approx,
                name="Model",
                line_color='blue',
                mode = 'lines',
                opacity=0.4))


In [442]:
def get_peaks(y, metrics): #determine the peaks of an array
    y = y.tolist()
    n = int(math.ceil(len(y)/T))
    step = 0
    x_peaks = []
    y_peaks = []
    for i in range(0,n):
        peak_index = y.index(metrics(y[step:step+T]))
        x_peaks.append(peak_index)
        y_peaks.append(y[peak_index])
        step = step+T
    return [x_peaks,y_peaks]

min_peaks = get_peaks(y,min)
max_peaks = get_peaks(y,max)

A = []
offset = []
for i in range(0, len(min_peaks[1])):
    c_a = (max_peaks[1][i] - min_peaks[1][i])/2
    c_offset = min_peaks[1][i] + c_a
    for j in range(0,T):
        A.append(c_a)
        offset.append(c_offset)
A = A[:-4]
offset = offset[:-4]

features = [*X,*A,*offset,len(X)]

def variable_sinusoid(features,omega,phase):
    R = []
    l = int(features[-1])
    for i in range(l):
        R.append(features[l+i]*np.sin(omega*features[i]+phase) + features[2*l+i])
    return R

def variable_get_p0(x, y): 
    phase0 = 0
    omega0 = 2.*np.pi/T
    return [omega0, phase0]        

In [443]:
param, covariance = curve_fit(variable_sinusoid, features, y, p0=variable_get_p0(X,y))
omega,phase = param

best_approx = variable_sinusoid(features,omega,phase)

fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Monthly sum of solar radiation (unit : kWh/m^2)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=df.index,
                y=df['GHIm'],
                name="Global horizontal irradiation",
                line={'dash': 'dash', 'color': 'red'},
                opacity=0.8))


fig.add_trace(go.Scatter(
                x=df.index,
                y=best_approx,
                name="Best model",
                line_color='blue',
                mode = 'lines',
                opacity=0.4))

## Predicting the future trendline

In [444]:
from sklearn import linear_model

# reshape x_peaks
x_min_peaks = list(map(lambda el:[el], min_peaks[0])) 
x_max_peaks = list(map(lambda el:[el], max_peaks[0]))

# min model
model_min = linear_model.LinearRegression()
model_min.fit(x_min_peaks,min_peaks[1])

# max model
model_max = linear_model.LinearRegression()
model_max.fit(x_max_peaks,max_peaks[1])

while (x_max_peaks[-1][0]<432):
    x_max_peaks.append([x_max_peaks[-1][0] + T])
    
while (x_min_peaks[-1][0]<432):    
    x_min_peaks.append([x_min_peaks[-1][0] + T])
y_pred_min = model_min.predict(x_min_peaks)
y_pred_max = model_max.predict(x_max_peaks)

x_min = [datetime.date(1994+e[0]//12, e[0]%12+1, 1) for e in x_min_peaks]
x_max = [datetime.date(1994+e[0]//12, e[0]%12+1, 1) for e in x_max_peaks]

In [445]:
fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Monthly sum of solar radiation (unit : kWh/m^2)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=df.index,
                y=df['GHIm'],
                name="Global horizontal irradiation",
                line={'dash': 'dash', 'color': 'red'},
                opacity=0.8))


fig.add_trace(go.Scatter(
                x=x_min,
                y=y_pred_min,
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x_max,
                y=y_pred_max,
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))

In [471]:
X_pred = np.arange(0, 440)

In [484]:
A_pred = A * 1
offset_pred = offset * 1
for i in range(len(min_peaks[1]),len(min_peaks[1])+10):
    c_a = (y_pred_max[i] - y_pred_min[i])/2
    c_offset = y_pred_min[i] + c_a
    for j in range(0,T):
        A_pred.append(c_a)
        offset_pred.append(c_offset)

features_pred = [*X_pred,*A_pred,*offset_pred,len(X_pred)]

In [486]:
best_approx = variable_sinusoid(features,omega,phase)
predict = variable_sinusoid(features_pred,omega,phase)

fig = go.Figure(layout = go.Layout(
        xaxis=dict(showgrid = True,title = "Year",color = 'black'),
        yaxis=dict(showgrid = True,title = "Monthly sum of solar radiation (unit : kWh/m^2)",color = 'black'), 
    ))

fig.add_trace(go.Scatter(
                x=df.index,
                y=df['GHIm'],
                name="Global horizontal irradiation",
                line={'dash': 'dash', 'color': 'red'},
                opacity=0.8))


fig.add_trace(go.Scatter(
                x=[datetime.date(1994+e//12, e%12+1, 1) for e in X_pred],
                y=predict,
                name="Best model",
                line_color='blue',
                mode = 'lines',
                opacity=0.4))

fig.add_trace(go.Scatter(
                x=x_min,
                y=y_pred_min,
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x_max,
                y=y_pred_max,
                line_color='blue',
                mode = 'lines+markers',
                opacity=0.8))